In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit, float64
import time
import os
import copy

In [2]:
def load_excel_data(directory):
    """
    Load all Excel files and their sheets from the specified directory into a list of dataframes.

    Parameters:
    directory (str): The path to the folder containing the Excel files.

    Returns:
    list: A list of pandas dataframes containing the data from each sheet in the Excel files.
    """
    # Initialize an empty list to store dataframes
    datasets = []
    # Iterate over each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(directory, filename)

            # Load the Excel file
            excel_file = pd.ExcelFile(file_path)

            # Iterate over each sheet in the Excel file
            for sheet_name in excel_file.sheet_names:
                # Load the sheet into a dataframe and append to the datasets list
                df = pd.read_excel(excel_file, sheet_name=sheet_name)
                datasets.append(df)

    return datasets


def save_datasets_to_excel(datasets, output_file):
    """
    Save a list of dataframes to a single Excel file with each dataframe in a separate sheet.

    Parameters:
    datasets (list): A list of tuples containing filename, sheet name, and dataframe.
    output_file (str): The path to the output Excel file.
    """
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        for i, (df) in enumerate(datasets):
            sheet_name_clean = f"Sheet_{i}"[
                :31
            ]  # Excel sheet names must be <= 31 chars
            df.to_excel(writer, sheet_name=sheet_name_clean, index=False)

In [3]:
# Example usage of the function
directory = "data/"
datasets = load_excel_data(directory)

In [4]:
# for i in range(len(datasets)):
#     plt.plot(datasets[i]['timestamp'], datasets[i]['trolley_position'], label='Trolley Position')
#     plt.plot(datasets[i]['timestamp'], datasets[i]['cable_length'], label='Cable Length')
#     plt.xlabel('Time')
#     plt.title(f"Data Set {i+1}")
#     plt.legend()
#     plt.show()

In [5]:
DURATION = 15  # duration in seconds
DT = 0.0001  # time increment in seconds
# Create a time array
time_array = np.arange(0, DURATION + DT, DT)
NUM_STEPS = len(time_array)

interpolated_datasets = []
for i in range(len(datasets)):
    new_trolley_position = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["trolley_position"]
    )
    new_cable_length = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["cable_length"]
    )
    new_sway_angle = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["sway_angle"]
    )
    new_trolley_motor_voltage = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["trolley_motor_voltage"]
    )
    new_hoist_motor_voltage = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["hoist_motor_voltage"]
    )

    interpolated_df = {
        "time": time_array,
        "trolley_position": new_trolley_position,
        "cable_length": new_cable_length,
        "sway_angle": new_sway_angle,
        "trolley_motor_voltage": new_trolley_motor_voltage,
        "hoist_motor_voltage": new_hoist_motor_voltage,
    }

    interpolated_datasets.append(interpolated_df)

In [6]:
# for i in range(len(interpolated_datasets)):
    # plt.plot(interpolated_datasets[i]['time'], interpolated_datasets[i]['trolley_position'], label='Trolley Position')
    # plt.plot(interpolated_datasets[i]['time'], interpolated_datasets[i]['cable_length'], label='Cable Length')
    # plt.xlabel('Time')
    # plt.title(f"Interpolated Data Set {i+1}")
    # plt.legend()
    # plt.show()

In [7]:
# Open the JSON file
with open("gantry_crane_parameters.json", "r") as file:
    data = json.load(file)

global all_parameters; all_parameters = data["gantry_crane_system_model"]["parameters"]
measured_parameters = {}
approximated_parameters = {}

for parameter in all_parameters:
    value = all_parameters[parameter]["value"]
    unit = all_parameters[parameter]["unit"]
    description = all_parameters[parameter]["description"]

    # print(f"{parameter}: {value} {unit} ({description})")

    if all_parameters[parameter]["measured"]:
        measured_parameters[parameter] = all_parameters[parameter]
    else:
        approximated_parameters[parameter] = all_parameters[parameter]

print(f'Number of approximated parameters: {len(approximated_parameters)}')

Number of approximated parameters: 13


In [8]:
from model import Simulator
import utility as util

simulator = Simulator(DT, NUM_STEPS)

In [9]:
def cost_function(params, data_arrs):
    """
    Cost function is the sum of the root mean squared errors between the interpolated data and the simulated data.
    """
    # Extract the parameters
    input_voltages = {
        "trolley_motor_voltage": data_arrs["trolley_motor_voltage"],
        "hoist_motor_voltage": data_arrs["hoist_motor_voltage"],
    }
    var_init_conditions = {
        "x": data_arrs["trolley_position"][0],
        "l": data_arrs["cable_length"][0],
        "theta": data_arrs["sway_angle"][0],
    }
    # print(f"Simulating with parameters: {params}")
    # print(f"Inital conditions: {var_init_conditions}")
    
    simulator.simulate(params, input_voltages, var_init_conditions)
    simulation_results = simulator.get_results()
    sum_RMSE = util.calculate_sum_root_mean_squared_errors(
        pd.DataFrame(simulation_results), pd.DataFrame(data_arrs)
    )
    if np.isnan(sum_RMSE):
        print(f"Sum RMSE is NaN")
        return np.inf
    
    print(f"\rSum RMSE: \t{round(sum_RMSE, 5)}", end="\r")

    return sum_RMSE

In [10]:
optimize_range = {
    "trolley_mass": (1.0, 5.0),  # Done
    "trolley_damping_coefficient": (0.1, 10.0),  # Done
    "cable_damping_coefficient": (0.1, 10.0),  # Done
    "trolley_motor_rotator_inertia": (0.00001, 0.001),  # Done
    "trolley_motor_damping_coefficient": (0.1, 10.0),
    "trolley_motor_back_emf_constant": (0.001, 0.1),  # Done
    "trolley_motor_torque_constant": (0.01, 1.0),  # Done
    "hoist_motor_rotator_inertia": (0.000001, 0.0001),  # Done
    "hoist_motor_damping_coefficient": (0.1, 10.0),
    "hoist_motor_back_emf_constant": (0.01, 1.0),  # Done
    "hoist_motor_torque_constant": (0.0001, 0.01),  # Done
    "trolley_motor_activation_threshold_voltage": (0.1, 10.0),  # Done
    "hoist_motor_activation_threshold_voltage": (0.1, 10.0),  # Done
}

### Gradient Descent

In [11]:
LEARNING_RATE = 0.01
MAX_VARIATIONS = 2
MAX_ITERATIONS = 1000
NUM_DATASETS = len(interpolated_datasets)
NUM_DATASETS = 3

In [12]:
best_parameters = copy.deepcopy(all_parameters)
best_average_total_cost = 0
for i in range(NUM_DATASETS):
    best_average_total_cost += cost_function(best_parameters, interpolated_datasets[i])
    print(f"Best Cost: {best_average_total_cost}", end="\r")

In [13]:
variation = 0
while variation <= MAX_VARIATIONS:
    current_parameters = copy.deepcopy(all_parameters)
    for parameter in approximated_parameters:
        current_parameters[parameter]["value"] = np.random.uniform(
            optimize_range[parameter][0], optimize_range[parameter][1]
        )

    print(f"Variation: {variation}")
    for parameter in approximated_parameters:
        print(f"{parameter}: {current_parameters[parameter]['value']}", end=" | ")
    print()

    diverge = False
    for iteration in range(MAX_ITERATIONS):     
        print(f"Iteration: {iteration}")

        # Loop through each parameter to optimize
        for parameter in approximated_parameters:
            if diverge:
                break

            # Loop through each dataset to calculate the cost and gradient
            total_parameter_gradient = 0
            for dataset_num in range(NUM_DATASETS):
                if diverge:
                    break

                print(f"Optimizing {parameter} for dataset {dataset_num}", end="\r")

                # Calculate the cost function
                cost = cost_function(current_parameters, interpolated_datasets[dataset_num])

                if cost == np.inf:
                    print(f"Cost is infinite")
                    diverge = True
                    break

                # Calculate the gradient
                new_parameters = copy.deepcopy(current_parameters)
                new_parameters[parameter]["value"] += 0.000001
                new_cost = cost_function(new_parameters, interpolated_datasets[dataset_num])
                gradient = (new_cost - cost) / 0.000001
                total_parameter_gradient += gradient
            
            # Update the parameter
            current_parameters[parameter]["value"] -= LEARNING_RATE * total_parameter_gradient

            if current_parameters[parameter]["value"] < 0:
                current_parameters[parameter]["value"] = optimize_range[parameter][0]

            print(f"{parameter} || Cost: {cost}, Gradient: {total_parameter_gradient}, New Value: {current_parameters[parameter]['value']}")
            print("-" * 100)
        
        if diverge:
            break
       
        else:
            average_total_cost = 0
            for dataset_num in range(NUM_DATASETS):    
                average_total_cost += cost_function(current_parameters, interpolated_datasets[dataset_num])
            average_total_cost /= NUM_DATASETS
            
            if average_total_cost < best_average_total_cost:
                best_average_total_cost = average_total_cost
                best_parameters = copy.deepcopy(current_parameters)
                print(f"New Best Cost: {best_average_total_cost}")
                print(best_parameters)

                # Save the best parameters to a JSON file
                with open("GD_best_gantry_crane_parameters.json", "w") as file:
                    json.dump(best_parameters, file, indent=4)

    if diverge:
        variation -= 1
        if variation < 0:
            variation = 0
        print("Variation Failed. Restarting current variation...")
        print("*" * 100)
        continue

    print(f"Results: Average Cost: {average_total_cost}, Best Average Cost: {best_average_total_cost}")
    print(f"Best Parameters:", end=" ")
    for parameter in approximated_parameters:
        print(f"{parameter}: {best_parameters[parameter]['value']}", end=" | ")
    print()
    print("*" * 100)

    variation += 1

Variation: 0
trolley_mass: 4.050215211908318 | trolley_damping_coefficient: 1.6895608831815085 | cable_damping_coefficient: 3.090401138818727 | hoist_motor_activation_threshold_voltage: 2.099740231925906 | hoist_motor_rotator_inertia: 5.780886972797849e-05 | hoist_motor_damping_coefficient: 1.2098164617234533 | hoist_motor_torque_constant: 0.007579946364295252 | hoist_motor_back_emf_constant: 0.49841704627158573 | trolley_motor_activation_threshold_voltage: 7.391908397419534 | trolley_motor_rotator_inertia: 8.33093015664635e-05 | trolley_motor_damping_coefficient: 8.76919844583819 | trolley_motor_torque_constant: 0.902455521836503 | trolley_motor_back_emf_constant: 0.005519118565804166 | 
Iteration: 0
trolley_mass || Cost: 0.14000700341202818, Gradient: -3.3681391009565687e-08, New Value: 4.050215212245132
----------------------------------------------------------------------------------------------------
trolley_damping_coefficient || Cost: 0.14000700341202818, Gradient: 0.0, New Valu

KeyboardInterrupt: 